In [1]:
import torch
import os
import numpy as np
import cv2
import mediapipe as mp
import pickle
import time
import re 
from deepface import DeepFace

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#DEVICE = torch.device("cpu")

c:\Users\ronsh\anaconda3\envs\cuda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dict_path = r"./Data/Phoenix14/"
poses_names = [("poses_train.pkl","train"), ("poses_val.pkl","dev"), ("poses_test.pkl","test")]
emotions_names = [("emotions_train.pkl","train"), ("emotions_val.pkl","val"), ("emotrion_test.pkl","test")]
raw_imgs_path = r".\Data\features\PHOENIX-2014-T-release-v3\PHOENIX-2014-T\features\fullFrame-210x260px"

**Extraction Functions**

In [30]:
   
def extract_poses_from_samples(poses = None, img_path = "", dict_name = "poses"):
    paths = os.listdir(img_path)
    paths = list(filter(lambda x: len(x.split(".")) == 1, paths))
    if poses is not None:
        paths = list(filter(lambda x: x not in poses.keys(),paths))
        print(f"number of folders processed is {len(poses.keys())}, nunber of folders left is {len(paths)}")

    else:
        poses = {}
   
    mp_pose = mp.solutions.pose
    for index, folder in enumerate(paths):
        print(folder)
        start = time.time()
        pose = mp_pose.Pose(min_detection_confidence = 0.3,min_tracking_confidence = 0.3, model_complexity=2,
                enable_segmentation=True)
        if index % 10 == 0:
            with open(r'./Data/Phoenix14/{name}'.format(name=dict_name), 'wb') as f:
                pickle.dump(poses, f, pickle.HIGHEST_PROTOCOL)
                print(f"folder index : {index} , saved!")
       
        poses[folder] = []
        imgs = os.listdir(img_path +"/" + folder)
       
        for img in imgs:
          frame = cv2.imread(img_path +"/" + folder + "/" + img)
          results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
          landmark_list = str(results.pose_landmarks).split("landmark")[1:]
          landmark_cord = [tuple(re.findall("-?\d.\d*", landmark)[:-1]) for landmark in landmark_list]
          poses[folder].append(landmark_cord)
       
        end = time.time()
        total_time = end - start
        print("\n"+ str(total_time))
    return poses

In [29]:
def extract_emotions_from_samples(emothins = None, path = "",num_of_emo = 7,dict_name = "emothin"):
  paths = os.listdir(path)
  paths = list(filter(lambda x: len(x.split(".")) == 1, paths))
  if emothins is not None:
    paths = list(filter(lambda x: x not in emothins.keys(),paths))
    print(f"number of folders processed is {len(emothins.keys())}, nunber of folders left is {len(paths)}")
  else:
    emothins = {}
  
  for index, folder in enumerate(paths):
    print(folder)
    if index % 10 == 0:
      with open(r'./Data/Phoenix14/{name}'.format(name=dict_name), 'wb') as f:
          pickle.dump(emothins, f, pickle.HIGHEST_PROTOCOL)
          print(f"folder index : {index} , saved!")

    emothins[folder] = []
    imgs = os.listdir(path +"/" + folder)
    for img in imgs:
      try:
        result = list(DeepFace.analyze(img_path = f"./{path}/{folder}/{img}", 
            actions = ['emotion'])["emotion"].values())
        result = torch.tensor(result) / 100
      except Exception as e:
        #print(e)
        result = torch.tensor([1/num_of_emo for _ in range(num_of_emo)])
     
      emothins[folder].append(result)
  return emothins 

**Extract Emotaions**

In [32]:
for dict_name , folder in emotions_names:
    
     cur_path = dict_path + dict_name
     if os.path.exists(cur_path):
          with open(cur_path, 'rb') as f:
               print("loading dict")
               emothins = pickle.load(f)
     
          emothins = extract_emotions_from_samples(emothins,path = os.path.join(raw_imgs_path,folder),dict_name=dict_name)
     else:
          emothins = extract_emotions_from_samples(path = os.path.join(raw_imgs_path,folder),dict_name=dict_name)

     with  open(cur_path, 'wb') as f:
           pickle.dump(emothins, f, pickle.HIGHEST_PROTOCOL)


loading dict
number of folders processed is 2, nunber of folders left is 0


**Extract Poses**

In [34]:
for dict_name , folder in poses_names:
   
     cur_path = dict_path + dict_name
     if os.path.exists(cur_path):
          with open(cur_path, 'rb') as f:
               print("loading dict")
               poses = pickle.load(f)
     
          poses = extract_poses_from_samples(poses,img_path = os.path.join(raw_imgs_path,folder),dict_name=dict_name)
     else:
          poses = extract_poses_from_samples(img_path = os.path.join(raw_imgs_path,folder),dict_name=dict_name)

     with  open(cur_path, 'wb') as f:
           pickle.dump(poses, f, pickle.HIGHEST_PROTOCOL)

loading dict
number of folders processed is 2, nunber of folders left is 0
